<img src="../../images/TGC_trans.png" alt="Enexis JADS" style="width:18%; float:right">

## Description

Parameterized model, taking the Enexis grid constraint into account.

### Parameters

#### Electric Vehicles

User entry via Mobile App
 
- ``duration of stay``: time that the EV will be parked. 
- ``Desired State of Charge``: Desired % of the battery capacity when leaving

Attributes of the Electric Vehicle:

- ``EV maximum power input``: constrainted by the EV - On Board Charger (OBC) (3.3; 7.36; 11; 22 kW)
- ``EV Current State of Charge``: Current % of the battery capacity charged
- ``EV Battery Capacity``: kWh total battery capacity

#### Infrastructure 

- ``Parking Lots``: number of parking lots
- ``EVSE's``: number of charging equipments (<= Parking Lots)
- ``EVSE max power output``: constrainted by the EVSE Type (Single Phase = 7.36 kW & Three-Phase = 22 kW)

#### Capacity Enexis

- ``Enexis maximum power output``: maximum kW which can be requested to the Enexis Energy Grid


## Purpose

- Verify and Validate known test cases

## Visualization

‼ 🚧 Note that at the animation is limited to two EVSE's currenty 🚧

<img src="images/fig_3.png" width="40%"/>
<img src="images/demo.png" width="30%"/>


# Simulation function for the e-parking

In [1]:
import salabim as sim
import pandas as pd


# ------------------------------------------------------------
# Simulation code for Tetris the Game Charger
# ------------------------------------------------------------
# function to run simulation
def sim_e_parking(
    inter_arr_time_distr,  # inter-arrival time distribution
    energy_request_distr,  # energy request distribution
    fixed_utilization,
    number_of_EVs,
    ev_mpi,
    ev_dos,
    number_of_PARKING_LOTs,
    number_of_EVSEs,  # number of EVSE's
    evse_mpo,
    enexis_mpo,
    sim_time,
    time_unit,
    random_seed,
    run,
    video_name = None,
):
    # ------------------------------------------------------------

    # Generator which creates EV's up to a certain number
    class EV_Generator(sim.Component):
        # setup method is called when the component is created
        # and is used to initialize the component
        # switch off monitoring for mode and status
        def setup(self, number_of_EVs):
            self.no_of_ev = number_of_EVs

        def process(self):
            ev = 1
            while ev <= self.no_of_ev:
                EV()
                iat = inter_arr_time_distr.sample()
                if fixed_utilization:
                    iat = iat / number_of_EVSEs
                self.hold(iat)
                ev += 1

    # EV component
    class EV(sim.Component):
        def setup(self):
            # energy request in kwh
            self.max_kw = ev_mpi
            self.kwh_req = energy_request_distr.sample()
            self.stay = ev_dos
            self.kwh_charged = 0
            self.evse_name = None

        def process(self):
            self.enter(waitingline)
            for LOT in e_parking_plots:
                if LOT.ispassive():
                    LOT.activate()
                    break  # activate at most one parking lot
            self.passivate()
            print(
                f"EV: {self.name()} charged: {100*self.kwh_charged/self.kwh_req}% station: {self.evse_name} left at: {app.now()}"
            )

    # Parking lot component (includes charging if available)
    class LOT(sim.Component):
        def setup(self):
            self.length = sim.Monitor(
                name="length", monitor=True, level=True, type="int32"
            )
            self.length_of_park = sim.Monitor(
                name="length_of_park", monitor=True, level=False, type="float"
            )
            self.length_of_charge = sim.Monitor(
                name="length_of_charge", monitor=True, level=False, type="float"
            )
            self.power_consumption = sim.Monitor(
                name="power.", monitor=True, level=True, type="float"
            )

            if EVSE_POOL.available_quantity() > 0:
                self.request(EVSE_POOL)
                self.evse = True
            else:
                self.evse = False
            self.power = evse_mpo

        def process(self):
            while True:
                # get an ev from the queue
                self.length.tally(0)
                while len(waitingline) == 0:
                    self.passivate()
                self.ev = waitingline.pop()
                self.length.tally(1)
                # determine power (if no EVSE then 0)
                power = min(
                    self.evse * self.ev.max_kw,  # if self.evse = false then 0
                    self.power,
                    ENEXIS_KW.available_quantity(),
                )
                if power > 0:
                    # request power from Eneixs
                    self.request((ENEXIS_KW, power))
                    # determine time to charge
                    time_charge_in_full = self.ev.kwh_req / power
                    time_constr_to_charge = min(time_charge_in_full, self.ev.stay)
                    # charging
                    self.power_consumption.tally(power)
                    tetris_game_charger.activate()
                    self.hold(time_constr_to_charge)
                    self.release((ENEXIS_KW, power))
                else:
                    # no charging possible
                    time_constr_to_charge = 0
                # update statistics
                self.ev.kwh_charged = time_constr_to_charge * power
                self.ev.evse_name = self.name()
                self.length_of_charge.tally(time_constr_to_charge)
                tetris_game_charger.activate()
                self.power_consumption.tally(0)
                # plain parking
                self.hold(self.ev.stay - time_constr_to_charge)
                self.ev.activate()

    # Tetris Game Charger - initial process under construction
    class TGC(sim.Component):
        def setup(self):
            self.total_power_consumption = sim.Monitor(
                name="total_power", monitor=True, level=True, type="float"
            )

        def process(self):
            while True:
                tot_pwr = sum(x.power_consumption.get() for x in e_parking_plots)
                self.total_power_consumption.tally(tot_pwr)
                self.passivate()

    # ------------------------------------------------------------
    # https://www.salabim.org/manual/Reference.html#environment
    app = sim.App(
        trace=False,  # defines whether to trace or not
        random_seed=random_seed,  # if “*”, a purely random value (based on the current time)
        time_unit=time_unit,  # defines the time unit used in the simulation
        name="Charging Station",  # name of the simulation
        do_reset=True,  # defines whether to reset the simulation when the run method is called
        yieldless=True,  # defines whether the simulation is yieldless or not
    )

    # Instantiate and activate the client generator
    EV_Generator(name="Electric Vehicles Generator", number_of_EVs=number_of_EVs)

    # Create Queue and set monitor to stats_only
    # https://www.salabim.org/manual/Queue.html
    waitingline = sim.Queue(name="Waiting EV's", monitor=True)
    waitingline.length.reset_monitors(stats_only=True)
    waitingline.length_of_stay.reset_monitors(stats_only=True)

    # Create EVSE Pool and ENEXIS connection as capacity Resources
    EVSE_POOL = sim.Resource("EVSE's", capacity=number_of_EVSEs)
    ENEXIS_KW = sim.Resource("ENEXIS", capacity=enexis_mpo)

    # Instantiate the EVSE's, list comprehension
    e_parking_plots = [LOT() for _ in range(number_of_PARKING_LOTs)]

    tetris_game_charger = TGC()

    app.AnimateMonitor(
        e_parking_plots[0].power_consumption,
        x=100,
        y=150,
        width=300,
        height=200,
        vertical_offset=0,
        vertical_scale=8,
        horizontal_scale=100,
        titlefont="Ubuntu Bold",
        titlefontsize=20,
        titlecolor="#B6D300",
        linewidth=5,
        linecolor="#B6D300",
        title="Power EVSE: 0",
        labels=list(range(0, enexis_mpo + 10 + 1, 5)),
        nowcolor="red",
    )

    app.AnimateMonitor(
        e_parking_plots[1].power_consumption,
        x=500,
        y=150,
        width=300,
        height=200,
        vertical_offset=0,
        vertical_scale=8,
        horizontal_scale=100,
        titlefont="Ubuntu Bold",
        titlefontsize=20,
        titlecolor="#B6D300",
        linewidth=5,
        linecolor="#B6D300",
        title="Power EVSE:1",
        labels=list(range(0, enexis_mpo + 10 + 1, 5)),
        nowcolor="red",
    )

    app.AnimateMonitor(
        tetris_game_charger.total_power_consumption,
        x=100,
        y=450,
        width=400,
        height=200,
        vertical_offset=0,
        vertical_scale=8,
        horizontal_scale=100,
        titlefont="Ubuntu Bold",
        titlefontsize=20,
        titlecolor="#D8006F",
        linewidth=5,
        linecolor="#D8006F",
        title="ENEXIS - Total Power Consumption",
        labels=list(range(0, enexis_mpo + 10 + 1, 5)),
        nowcolor="red",
    )

    sim.AnimateImage(
        image="../../images/TGC_green.png",
        # alpha=100,
        x=600,
        y=400,
        width=250,
        layer=0,
        angle=0,
    )

    app.animate(True)
    app.modelname("Tetris: the game charger")

    # Execute Simulation
    if video_name is not None:
        app.video("video/" + video_name + ".mp4")
        app.video_mode("2d")
        app.run(till=sim_time)
        app.video_close()
    else:
        app.run(till=sim_time)

    # lmbda = cnv_hr_to_mins / inter_arr_time_distr.mean()
    # if fixed_utilization:
    #     lmbda = lmbda * number_of_EVSEs

    # Return results
    return {
        "iat": inter_arr_time_distr.mean(),
        "pwr": energy_request_distr.mean(),
        # "run": run,
        # "lambda": lmbda,
        # "mu": cnv_hr_to_mins / energy_request_distr.mean(),
        "lots": number_of_PARKING_LOTs,
        "evse": number_of_EVSEs,
        # "RO": total_evse_lngt / number_of_EVSEs,
        "P0": 0,
        "Lq": waitingline.length.mean(),
        "Wq": waitingline.length_of_stay.mean(),
        # "Ls": total_evse_lngt + waitingline.length.mean(),
        # "Ws": total_evse_stay + waitingline.length_of_stay.mean(),
    }

# User parameters

In [2]:
# ------------------------------------------------------------
# Electric Vehicles
# ------------------------------------------------------------

# User entry via Mobile App 'duration of stay' and 'Desired State of Charge' (ev_dsc)
EV_DURATION_OF_STAY = 1.0  # hours
EV_DESIRED_STATE_OF_CHARGE = 1  # percentage of the battery capacity (ev_dsc)

# Electric Vehicles - attributes
EV_MAX_POWER_INPUT = (
    7.36  # constrainted by the EV - On Board Charger (OBC) (3.3; 7.36; 11; 22 kW)
)
EV_CURRENT_STATE_OF_CHARGE = (
    0.0  # percentage of the battery capacity being charged (ev_csc)
)
EV_BATTERY_CAPACITY =  69.2 # kWh total battery capacity (ev_bcp) ev-database.org
avg_energy__request = EV_BATTERY_CAPACITY * (
    EV_DESIRED_STATE_OF_CHARGE - EV_CURRENT_STATE_OF_CHARGE
)  # KW requested per EV
ENERGY_REQUEST_DISTR = sim.Uniform(avg_energy__request, avg_energy__request)

# ------------------------------------------------------------
# Infrastructure 
# ------------------------------------------------------------

# Parking Lots
NUMBER_OF_LOTS = 2

# EVSE - attributes
NUMBER_OF_EVSES = 2
EVSE_MAX_POWER_OUTPUT = 7.36  # constrainted by the EVSE Type (SP=7.36 kW & TP = 22 kW)

# Capacity Enexis
ENEXIS_MAX_POWER_OUTPUT = 10 # kW

# ------------------------------------------------------------
# Simulation parameters
# ------------------------------------------------------------

# Arrival pattern
NUMBER_OF_EVS = 16  # number of EV's to be simulated
FIXED_UTILIZATION = True  # increases the arrival rate with the number of EVSE's
ev_arrival_time = 0  # 10 / 7  # hours between two arrivals
INTER_ARR_TIME_DISTR = sim.Uniform(ev_arrival_time, ev_arrival_time)

# salabim parameters
SIM_TIME = None
TIME_UNIT = "hours"
RANDOM_SEED = "*"
RUN = 1

# Run

In [3]:
result = sim_e_parking(
    INTER_ARR_TIME_DISTR,  # inter-arrival time distribution
    ENERGY_REQUEST_DISTR,  # energy request distribution
    FIXED_UTILIZATION,  # increases the arrival rate with the number of EVSE's
    NUMBER_OF_EVS,  # number of EV's to be simulated
    EV_MAX_POWER_INPUT,  # constrainted by the EV - On Board Charger (OBC) (3.3; 7.36; 11; 22 kW)
    EV_DURATION_OF_STAY,  # hours that an EV stays at the parking lot
    NUMBER_OF_LOTS,  # number of parking lots
    NUMBER_OF_EVSES,  # number of EVSE's
    EVSE_MAX_POWER_OUTPUT,  # constrainted by the EVSE Type (SP=7.36 kW & TP = 22 kW)
    ENEXIS_MAX_POWER_OUTPUT,  # Maximum power output of Enexis Connection to the grid
    SIM_TIME,  # simulation time in time units
    TIME_UNIT,  # time unit
    RANDOM_SEED,  # random seed
    RUN,  # run number
)

print(pd.DataFrame(result, index=[0]))

EV: ev.1 charged: 10.635838150289016% station: lot.1 left at: 1.0
EV: ev.2 charged: 3.8150289017341033% station: lot.2 left at: 1.0
EV: ev.3 charged: 10.635838150289016% station: lot.1 left at: 2.0
EV: ev.4 charged: 3.8150289017341033% station: lot.2 left at: 2.0
EV: ev.5 charged: 10.635838150289016% station: lot.1 left at: 3.0
EV: ev.6 charged: 3.8150289017341033% station: lot.2 left at: 3.0
EV: ev.7 charged: 10.635838150289016% station: lot.1 left at: 4.0
EV: ev.8 charged: 3.8150289017341033% station: lot.2 left at: 4.0
EV: ev.9 charged: 10.635838150289016% station: lot.1 left at: 5.0
EV: ev.10 charged: 3.8150289017341033% station: lot.2 left at: 5.0
EV: ev.11 charged: 10.635838150289016% station: lot.1 left at: 6.0
EV: ev.12 charged: 3.8150289017341033% station: lot.2 left at: 6.0
EV: ev.13 charged: 10.635838150289016% station: lot.1 left at: 7.0
EV: ev.14 charged: 3.8150289017341033% station: lot.2 left at: 7.0
EV: ev.15 charged: 10.635838150289016% station: lot.1 left at: 8.0
EV: 